In [ ]:
# Install required packages
!pip install -qU langchain==0.3.12 langchain-chroma langchain-community pypdf langchain-openai wikipedia langgraph

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Environment setup
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# PDF Load

In [ ]:
# PDF Loading
from langchain.document_loaders import PyPDFLoader
pdf_files = ["Meditation_Techniques.pdf", "how-to-meditate.pdf"]
all_documents = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    docs = loader.load()
    all_documents.extend(docs)
documents = all_documents
print(f"Loaded {len(documents)} documents from {len(pdf_files)} PDFs.")

Loaded 39 documents from 2 PDFs.


In [ ]:
len(documents)

39

# Text Split

In [ ]:
# Text Splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
print(f"Split into {len(texts)} chunks.")

Split into 89 chunks.


# Embedding

In [ ]:
# Embedding
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Vector Store

In [ ]:
# Vectorstore (ChromaDB)
from langchain_chroma import Chroma as ch
persist_directory = "chroma_db"
db = ch.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=persist_directory
)

# Retriever

In [ ]:
# Retriever with similarity score threshold
retriever = db.as_retriever(
    search_type="similarity", #change
    search_kwargs={"k": 4}
)

In [ ]:
# RAG Prompt
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chat_models import ChatOpenAI

custom_prompt_template = """
Use ONLY the pieces of information provided in the context to answer the user's question.
If the answer is not present in the context, explicitly return: I don't know.
Do NOT try to make up an answer or use outside knowledge.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(custom_prompt_template)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# RAG LCEL chain
rag_chain = (
    {
        "context": lambda x: retriever.invoke(x["question"]),
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke({"question": "What is meditation?"})

'Meditation is commonly described as a training of mental attention that awakens us beyond the conditioned mind and habitual thinking, and reveals the nature of reality. It is understood as Natural Presence, which is a mindful, clear recognition of what is happening—here, now—and the open, allowing space that includes all experience.'

In [ ]:
rag_chain.invoke({"question": "How to do breathing exercises?"})

"To do breathing exercises, you can follow these steps:\n\n1. **Proper Breathing**: Breathe in slowly and quietly, so that a tiny thread placed in front of your nose does not move. Breathe out even more slowly than you breathed in. If possible, leave a short pause between your exhalation and the next inhalation. You can hold your breath for a few seconds if it's comfortable.\n\n2. **Life-Energy in the Chakras**: Breathe in and hold your breath at the third eye for a couple of seconds, then at the heart center during the second inhalation.\n\n3. **One-Four-Two Breathing**: Inhale for one count while repeating a name or mantra, hold your breath for four counts repeating the same name four times, and exhale for two counts repeating the name or mantra twice.\n\n4. **Gradual Increase**: As you progress, you can try breathing in for four counts, holding for sixteen, and breathing out for eight, but do this gradually.\n\n5. **Alternate Nostril Breathing**: Press the right nostril with your th

# Agent 1

In [ ]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain.agents import create_tool_calling_agent, AgentExecutor
# from langchain_community.tools import WikipediaQueryRun
# from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools import Tool

# Define a RAG tool that takes a query and returns the RAG answer
# def is_rag_no_answer(result):
#     result_str = str(result).strip().lower()
#     return (
#         result_str in ["i don't know", "i do not know", "not present", "no relevant information", "sorry", ""]
#         or len(result_str) < 10
#     )

class RAGTool:
    def __init__(self, qa_chain):
        self.qa_chain = qa_chain
    def __call__(self, input_dict) -> str:
        # Accepts a dict, extracts the 'query' key
        query = input_dict["query"] if isinstance(input_dict, dict) else input_dict
        # Always pass as {"question": ...} to the chain
        result = self.qa_chain.invoke({"question": query})
        # Normalize and check for "I don't know" or similar
        result_str = str(result).strip().lower()
        if result_str in [
            "i don't know", "i do not know", "not present", "no relevant information", "sorry", ""
        ] or len(result_str) < 10:
            return "__RAG_NO_ANSWER__"
        return result

rag_tool = Tool(
    name="RAG",
    func=RAGTool(rag_chain),
    description="Use this tool to answer questions using the provided PDF documents. Always try this tool first. If it returns __RAG_NO_ANSWER__, then try other tools."
)

# api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
# wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

tools = [rag_tool]

# Custom agent logic: always call RAG first, if it returns __RAG_NO_ANSWER__, then call Wikipedia
key = userdata.get('OPENAI_API_KEY')
llm = ChatOpenAI(api_key=key, model="gpt-4o-mini")

class ResearchResponse(BaseModel):
    topic: str
    answer: str
    sources: list[str]
    tools_used: list[str]

parser = PydanticOutputParser(pydantic_object=ResearchResponse)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            '''
            You are a research assistant with access to two tools:
            - "RAG": Answers questions using the provided PDF documents. Always try this tool first.
              If it returns "__RAG_NO_ANSWER__", then return I don't know.
            '''
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{query}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
).partial(format_instructions=parser.get_format_instructions())
agent = create_tool_calling_agent(
    llm=llm,
    prompt=prompt,
    tools=tools
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)



In [ ]:
# Now, just call the agent_executor. The agent will decide which tool(s) to use.
user_query = input("What can I help you research? ")
response = agent_executor.invoke({"query": user_query})
print(response)
#explain the distance from the main sequence and its formula.

What can I help you research? give me breathing exerciese steps


> Entering new AgentExecutor chain...

Invoking: `RAG` with `breathing exercises steps`


I don't know.I don't know.

> Finished chain.
{'query': 'give me breathing exerciese steps', 'output': "I don't know."}


# RAG Agent 2

In [ ]:
from langchain.tools import tool
from typing import List
import requests
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.chat_models import ChatOpenAI

# RAG Prompt Template
custom_prompt_template = """
Use ONLY the pieces of information provided in the context to answer the user's question.
If the answer is not present in the context, explicitly return: I don't know.
Do NOT try to make up an answer or use outside knowledge.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(custom_prompt_template)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# RAG Chain
rag_chain = (
    {
        "context": RunnableLambda(lambda x: retriever.invoke(x["question"])),
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

# Define tool
@tool
def rag_tool(question: str) -> str:
    """
    Answers a question using a retrieval-augmented generation (RAG) approach.
    Retrieves relevant documents from a vector store and uses an LLM to generate the final answer.
    """
    return rag_chain.invoke({"question": question})


In [ ]:
rag_chain.invoke({"question": "How to do breathing exercises?"})

"To do breathing exercises, you can follow these steps:\n\n1. **Basic Breathing**: Breathe in slowly and quietly, ensuring that a tiny thread placed in front of your nose does not move. Breathe out even more slowly than you inhaled. If possible, leave a short pause between exhalation and inhalation. You can hold your breath for a few seconds if it's comfortable.\n\n2. **Life-Energy in Chakras**: Breathe in and hold your breath at the third eye for a couple of seconds, then at the heart center during your next inhalation.\n\n3. **One-Four-Two Breathing**: Inhale for one count while repeating a name or mantra, hold your breath for four counts repeating the same name, and exhale for two counts repeating the name or mantra again.\n\n4. **Alternate Nostril Breathing**: Press the right nostril with your thumb and inhale through the left nostril while repeating a name. Hold your breath for four counts, then switch to exhale through the right nostril while repeating the name twice. Repeat this

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

meditation_assistant = create_react_agent(
    model=model,
    tools=[rag_tool],
    prompt="You are a helpful Meditation assistant. Your job is to use RAG tool to Answers questions using the provided PDF documents. ",
    name="meditation_assistant"
)


In [ ]:
from langchain_core.messages import (
    HumanMessage,
)
response = meditation_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="How to do breathing exercises?")
            ]
    }
    )
response["messages"]

[HumanMessage(content='How to do breathing exercises?', additional_kwargs={}, response_metadata={}, id='2c02a15f-1ac6-40d2-ae14-76ad2f6ad371'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Zm1EXojvK5ZhvsQYLo8HTUOf', 'function': {'arguments': '{"question":"How to do breathing exercises?"}', 'name': 'rag_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 102, 'total_tokens': 122, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BiBIUtPoSrlC4hYYgKhR3CW4EHagf', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='meditation_assistant', id='run--028b0273-f478-41dc-b45b-1b734b9f91c0-0', tool_calls=[{'name': 'rag_tool'

In [ ]:
response["messages"][-1].content

'To perform breathing exercises, you can follow these steps:\n\n1. **Basic Breathing**:\n   - Inhale slowly and quietly through your nose, ensuring that a tiny thread placed in front of your nose does not move.\n   - Exhale even more slowly than you inhaled, allowing for a short pause between exhaling and your next inhalation.\n   - If comfortable, you can hold your breath for a few seconds between breaths.\n\n2. **Life-Energy Breathing**:\n   - Inhale and hold your breath at the third eye (forehead center) for a couple of seconds, focusing your concentration there.\n   - On your next inhalation, hold your breath at the heart center.\n\n3. **One-Four-Two Breathing**:\n   - Inhale for one count while repeating a name or mantra.\n   - Hold your breath for four counts, repeating the same name or mantra four times.\n   - Exhale for two counts, repeating the name or mantra twice.\n\n4. **Gradual Breathing**:\n   - As you advance, you can try inhaling for four counts, holding for sixteen cou

In [ ]:
from langchain_core.messages import (
    HumanMessage,
)
response = meditation_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="How to know about natural presence?")
            ]
    }
    )
response["messages"]

[HumanMessage(content='How to know about natural presence?', additional_kwargs={}, response_metadata={}, id='97cc1ee7-596e-4d4b-9728-62c8664bd819'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bNgWEW2JW0rZJs0rtkSdAljm', 'function': {'arguments': '{"question":"What is natural presence?"}', 'name': 'rag_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 103, 'total_tokens': 122, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BiBJs170UQypGl9WEwZLM1vhH6XG2', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='meditation_assistant', id='run--54da76b7-2dc3-44b7-8a7b-bde40fc6786a-0', tool_calls=[{'name': 'rag_tool'

In [ ]:
response["messages"][-1].content

'Natural presence refers to a mindful and clear recognition of what is happening in the present moment—here and now. It embodies an open space that includes all experiences and is often described as the loving awareness that is our essence. In this state, you simply recognize what is arising, such as thoughts, feelings, sounds, and emotions, and allow life to unfold as it is.'

In [ ]:
from langchain_core.messages import (
    HumanMessage,
)
response = meditation_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="List out common issues for meditators")
            ]
    }
    )
response["messages"]

[HumanMessage(content='List out common issues for meditators', additional_kwargs={}, response_metadata={}, id='63b59bb7-d31d-4549-a4f5-4ec1fd20fc1e'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_T8Ddu3QgMjiJ6DzoLvT82NlE', 'function': {'arguments': '{"question":"What are common issues faced by meditators?"}', 'name': 'rag_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 103, 'total_tokens': 126, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BiBL5pX4u8rS8ebtNXs60M4CsAIZd', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='meditation_assistant', id='run--9a15991f-bf1a-4f47-a0d3-4ddc552779fa-0', tool_calls=

In [ ]:
response["messages"][-1].content

"Common issues faced by meditators include:\n\n1. **Getting Lost in Thought**: Many meditators struggle with an active mind, leading to distractions and a loss of focus during meditation.\n\n2. **Difficult Emotional Energies**: Emotions such as vulnerability, fear, or anger may surface, which can be challenging to deal with.\n\n3. **Physical Pain**: Discomfort from sitting still, particularly for those unaccustomed to the posture, or due to existing injuries or health conditions.\n\n4. **The Five Classic Challenges (Hindrances)**:\n   - **Grasping**: A desire for something more or different from the present moment.\n   - **Aversion**: Feelings of fear, anger, or a tendency to push away experiences.\n   - **Restlessness**: Agitation or jumpy energy that disrupts focus.\n   - **Sloth and Torpor**: Experiences of sleepiness or a heavy, lethargic state of mind and body.\n   - **Doubt**: Questions about the effectiveness of meditation and one's practice.\n\nThese issues are not necessarily 

In [ ]:
from langchain_core.messages import (
    HumanMessage,
)
response = meditation_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="How to chant mantras? Give me an example mantra and its name.")
            ]
    }
    )
response["messages"]

[HumanMessage(content='How to chant mantras? Give me an example mantra and its name.', additional_kwargs={}, response_metadata={}, id='28bffd1e-8f38-43fb-bc03-ee02e61b6a16'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zmFVVNWSjvU2Nalk1h6vRe7A', 'function': {'arguments': '{"question":"How to chant mantras? Give me an example mantra and its name."}', 'name': 'rag_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 111, 'total_tokens': 140, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BiBND01tyRw6Hbzue0IWrEC2xUPAz', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='meditation_assistant', id='run--e4599273-0

In [ ]:
response["messages"][-1].content

'To chant mantras, you can inwardly repeat words while feeling their essence in your heart. An example of a mantra is the **Gayatri Mantra**, which is considered one of the most powerful mantras according to the Bhagavad Gita.'

In [ ]:
from langchain_core.messages import (
    HumanMessage,
)
response = meditation_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="Give the Gayatri mantra and its meaning.")
            ]
    }
    )
response["messages"]

[HumanMessage(content='Give the Gayatri mantra and its meaning.', additional_kwargs={}, response_metadata={}, id='82cd736b-177b-4cab-afaf-5e59d1ca5ba8'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_lirH7ytyWgxJQe1oYST6ZZYy', 'function': {'arguments': '{"question":"What is the Gayatri mantra and what does it mean?"}', 'name': 'rag_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 105, 'total_tokens': 131, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BiBOIJWWpD3Cz5M0mPBKnRTIhKKrb', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='meditation_assistant', id='run--43651034-47c1-41d8-a780-9dde9aefb434-0', to

In [ ]:
response["messages"][-1].content

'The Gayatri mantra is a revered hymn found in the Bhagavad Gita, known for its spiritual significance. The mantra is:\n\nAum bhur bhuvah svah  \nTat savitur varenyam  \nBhargo devasya dhimahi  \nDhiyo yo nah prachodayat  \n\n### Meaning:\n"We meditate on the transcendental glory of the Deity Supreme, Who is inside the heart of the earth, inside the life of the sky, and inside the soul of the Heaven. May He stimulate and illumine our minds."\n\nReciting this mantra with devotion has led many spiritual seekers to attain perfection.'

# Class Local

# Meditation RAG Tool. PY

# Run_meditation_RAG agent.py

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from agents.email_agent import EmailAgent
def main():
    agent = EmailAgent()
    query = "Send an email on physical fitness to shashwotpradhan@gmail.com and rabindratamangstudy@gmail.com subject should be 💪 Prioritize Your Health: The Power of Physical Fitness. Email is from mindfuelnepal@gmail.com"
    result = agent.run(query)
    print(result)

if __name__ == "__main__":
    main()

# Meditation RAG Agent py

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from tools.email_tools import send_email_via_sendgrid
from config.setting import OPENAI_API_KEY

class EmailAgent:
    def __init__(self):
        self.openai_api_key = OPENAI_API_KEY
        self.model = ChatOpenAI(api_key=self.openai_api_key, model="gpt-4o-mini", verbose=True)
        self.agent = create_react_agent(
            model=self.model,
            tools=[send_email_via_sendgrid],
            prompt = (
              "You are a helpful Email assistant. Your job is to help users send emails."
              "based on their requests. Use the tool to send emails to respective people."

            ),
            name="email_assistant"
        )

    def run(self, user_input):
        response = self.agent.invoke({
            "messages": [HumanMessage(content=user_input)]
        }, config={"recursion_limit": 50})
        return response["messages"][-1].content